# 1-2 Create spare matrix

In [2]:
import pandas as pd

In [3]:
movies_raw = pd.read_csv('./data/movies_big.csv', dtype={'movieId': 'int32'})
ratings_raw = pd.read_csv('./data/ratings_big.csv', usecols=['userId', 'movieId', 'rating'], dtype={'movieId': 'int32', 'userId':'int32', 'rating':'float32'})

In [4]:
avarage_rating = ratings_raw.groupby('movieId')['rating'].mean()
avarage_rating.info()

<class 'pandas.core.series.Series'>
Int64Index: 53889 entries, 1 to 193886
Series name: rating
Non-Null Count  Dtype  
--------------  -----  
53889 non-null  float32
dtypes: float32(1)
memory usage: 631.5 KB


In [5]:
movies_with_rating_over_x = pd.DataFrame(avarage_rating).query('rating > 3.5').index
movies_with_rating_over_x

Int64Index([     1,      6,     11,     16,     17,     21,     25,     26,
                28,     29,
            ...
            193765, 193785, 193795, 193799, 193801, 193835, 193843, 193868,
            193872, 193874],
           dtype='int64', name='movieId', length=13466)

In [6]:
ratings_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27753444 entries, 0 to 27753443
Data columns (total 3 columns):
 #   Column   Dtype  
---  ------   -----  
 0   userId   int32  
 1   movieId  int32  
 2   rating   float32
dtypes: float32(1), int32(2)
memory usage: 317.6 MB


In [7]:
ratings_cleaned = ratings_raw.query(f'movieId == {list(movies_with_rating_over_x)}')
ratings_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16314048 entries, 0 to 27753443
Data columns (total 3 columns):
 #   Column   Dtype  
---  ------   -----  
 0   userId   int32  
 1   movieId  int32  
 2   rating   float32
dtypes: float32(1), int32(2)
memory usage: 311.2 MB


In [10]:
ratings_pivot_table = ratings_cleaned.pivot(
    index="movieId", columns="userId", values="rating"
).fillna(0);

In [12]:
from scipy.sparse import csr_matrix
ratings_csr_matrix = csr_matrix(ratings_pivot_table.values)
ratings_csr_matrix

<13466x278651 sparse matrix of type '<class 'numpy.float32'>'
	with 16314048 stored elements in Compressed Sparse Row format>

In [16]:
from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=5)
model_knn.fit(ratings_csr_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [18]:
from fuzzywuzzy import process


def recomender(movie_name, data, model, number_of_recommendations):
    model.fit(data)
    index = process.extractOne(movie_name, movies_raw["title"])[2]
    print(f'Movie selected: {movies_raw["title"][index]}\nIndex: {index}')
    print("Searching for recommendations...")
    distances, indices = model.kneighbors(
        data[index], n_neighbors=number_of_recommendations
    )
    for match_index in indices:
        print(movies_raw["title"][match_index].where(match_index != index))


recomender("toy story", ratings_csr_matrix, model_knn, 100)

Movie selected: Toy Story (1995)
Index: 0
Searching for recommendations...
0                                                     NaN
78                                      Juror, The (1996)
1005                         Three Caballeros, The (1945)
464     Englishman Who Went Up a Hill But Came Down a ...
139                                  Birdcage, The (1996)
                              ...                        
947                           Angel and the Badman (1947)
1532                                       Contact (1997)
23                                          Powder (1995)
690                                          Faces (1968)
410                              Air Up There, The (1994)
Name: title, Length: 100, dtype: object
